## Importe de paquetes

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from collections import Counter
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

AttributeError: module 'numpy' has no attribute 'ndarray'

    ## Configuración

### Configuración de la sesion Spark

In [2]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar../mysql-connector-java-8.0.28.jar'

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

22/09/05 09:18:06 WARN Utils: Your hostname, Ricardos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
22/09/05 09:18:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/05 09:18:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Configuración de la conexión a la BD

In [4]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = 'Estudiante_4_202214'
db_psswd = 'NT8ZK03Q4H'

PATH='./'

In [ ]:
sql_aeropuertos = 'ProyectoTransaccional.aeropuertosCopia'

In [ ]:
sql_vuelos = 'ProyectoTransaccional.vuelosCopia2'

In [ ]:
sql_proyecciones = 'ProyectoTransaccional.proyeccionesCopia'

In [ ]:
sql_divipola = 'ProyectoTransaccional.divipolaCopia'

In [ ]:
sql_pib = 'ProyectoTransaccional.PIBCopia'

## Cargue de datos

In [6]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

### Tabla aeropuertosCopia

In [ ]:
aeropuertos = obtener_dataframe_de_bd(db_connection_string, sql_aeropuertos, db_user, db_psswd)

In [8]:
aeropuertos.show(5)

+-----+----+--------------------+--------------------+------------+----------+-------+--------+-----------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+--------+--------------------+----------------+-------------+
|sigla|iata|              nombre|           municipio|departamento| categoria|latitud|longitud|      propietario|          explotador|longitud_pista|ancho_pista|  pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|    tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+--------------------+--------------------+------------+----------+-------+--------+-----------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+--------+--------------------+----------------+-------------+
|  9cg|    |san jose del ariporo|      Paz de Ariporo|    casanare|AerÃ³dromo|    500|-70.0

### Tabla vuelosCopia

In [ ]:
vuelos = obtener_dataframe_de_bd(db_connection_string, sql_vuelos, db_user, db_psswd)

In [ ]:
vuelos.show(5)

### Tabla proyeccionesCopia

In [ ]:
proyecciones = obtener_dataframe_de_bd(db_connection_string, sql_proyecciones, db_user, db_psswd)

In [ ]:
proyecciones.show(5)

### Tabla divipolaCopia

In [ ]:
divipola = obtener_dataframe_de_bd(db_connection_string, sql_divipola, db_user, db_psswd)

In [ ]:
divipola.show(5)

### Tabla pibCopia

In [7]:
pib = obtener_dataframe_de_bd(db_connection_string, sql_pib, db_user, db_psswd)

In [ ]:
pib.show(5)

# Perfilamiento de los datos

El grupo de consultores de analitica descriptiva de Wide World Importers asegura que las tablas Grupo de compras, Ciudades, Colores, Paises, Clientes, Categorias de clientes, Paquetes, Estados y provincias, Productos y Personas ya fueron perfiladas y corregidos; sin embargo le recomiendan revisar el diccionario suministrado dado que todas las tablas serán utilizadas en futuros requerimientos.
Si tienen alguna duda relacionada con estas tablas deben incluirla en las preguntas o aclaraciones que le entreguen a la organización como parte del resultado de esta fase de entendimiento.

En este punto de la consultorìa, nuestra responsabilidad es analizar la tabla movimientosCopia, que registra el movimiento en inventario de un producto. La empresa no compartió información de las tablas de clientes ni de proveedores para esta actividad. Con respecto a esta fuente nos comentan lo siguiente:

- La cantidad máxima de productos movidos es 50 millones por transacción
- Existen 236.668 movimientos de producto realizados desde 2013
- Los movimientos están relacionados con 228.265 clientes
- El formato de fechas manejado es YYYY-MM-DD HH:MM:SS
- Actualmente tenemos 13 proveedores


## ¿Cuantos datos nos compartieron para el proyecto?

### Tabla aeropuertosCopia

In [9]:
print(f"registros = {aeropuertos.count()}", f"columnas = {len(aeropuertos.columns)}")

registros = 547 columnas = 23


In [10]:
aeropuertos.schema

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,IntegerType,true),StructField(longitud,StringType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,IntegerType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(orientacion,StringType,true),StructField(elevacion,StringType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))

In [11]:
Counter([col[1] for col in aeropuertos.dtypes])

Counter({'string': 17, 'int': 4, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla vuelosCopia

In [9]:
print(f"registros = {vuelos.count()}", f"columnas = {len(vuelos.columns)}")

registros = 547 columnas = 23


In [10]:
vuelos.schema

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,IntegerType,true),StructField(longitud,StringType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,IntegerType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(orientacion,StringType,true),StructField(elevacion,StringType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))

In [11]:
Counter([col[1] for col in vuelos.dtypes])

Counter({'string': 17, 'int': 4, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla proyeccionesCopia

In [9]:
print(f"registros = {proyecciones.count()}", f"columnas = {len(proyecciones.columns)}")

registros = 547 columnas = 23


In [10]:
proyecciones.schema

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,IntegerType,true),StructField(longitud,StringType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,IntegerType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(orientacion,StringType,true),StructField(elevacion,StringType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))

In [11]:
Counter([col[1] for col in proyecciones.dtypes])

Counter({'string': 17, 'int': 4, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla divipolaCopia

In [ ]:
print(f"registros = {divipola.count()}", f"columnas = {len(divipola.columns)}")

In [ ]:
divipola.schema

In [ ]:
Counter([col[1] for col in divipola.dtypes])

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla pibCopia

In [9]:
print(f"registros = {pib.count()}", f"columnas = {len(pib.columns)}")

registros = 547 columnas = 23


In [10]:
pib.schema

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,IntegerType,true),StructField(longitud,StringType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,IntegerType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(orientacion,StringType,true),StructField(elevacion,StringType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))

In [11]:
Counter([col[1] for col in pib.dtypes])

Counter({'string': 17, 'int': 4, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

## ¿Qué es una fila de cada fuente compartida?

### Tabla aeropuertosCopia

In [ ]:
aeropuertos.where(aeropuertos['sigla'] == '7gv').show()

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla vuelosCopia

In [ ]:
vuelos.show(1)

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla proyeccionesCopia

In [ ]:
proyecciones.show(1)

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla divipolaCopia

In [ ]:
divipola.show(1)

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla pibCopia

In [ ]:
pib.show(1)

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

## Análisis descriptivo

### Tabla aeropuertosCopia

In [13]:
aeropuertos.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


In [15]:
reporte = ProfileReport(aeropuertos.toPandas())
reporte

NameError: name 'ProfileReport' is not defined

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla vuelosCopia

In [13]:
vuelos.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


In [15]:
reporte = ProfileReport(vuelos.toPandas())
reporte

NameError: name 'ProfileReport' is not defined

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla proyeccionesCopia

In [13]:
proyecciones.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


In [15]:
reporte = ProfileReport(proyecciones.toPandas())
reporte

NameError: name 'ProfileReport' is not defined

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla divipolaCopia

In [13]:
divipola.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


In [15]:
reporte = ProfileReport(divipola.toPandas())
reporte

NameError: name 'ProfileReport' is not defined

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla pibCopia

In [13]:
pib.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


In [15]:
reporte = ProfileReport(pib.toPandas())
reporte

NameError: name 'ProfileReport' is not defined

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

# Análisis de calidad de datos

## Completitud

### Tabla aeropuertosCopia

In [17]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter(df[c].isNull()).count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados


In [18]:
contar_vacios(movimientos)

[]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [19]:
def contar_blancos(df):
    resultados = []
    for c in df.columns:
        vacios = df.where(df[c] == '').count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados


In [20]:
contar_blancos(movimientos)

número de vacíos para columna ProveedorID: 197182


número de vacíos para columna OrdenDeCompraID: 197182


[197182, 197182]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [21]:
def cardinalidad(df):
    resultado = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
    return resultado


In [22]:
columnas_alta_cardinalidad_movimientos = cardinalidad(movimientos)
columnas_alta_cardinalidad_movimientos

{'TransaccionProductoID': 173659}

Respecto a cardinalidad, solo la columna TransaccionProductoID tiene una cardinalidad superior al 50%.

### Tabla vuelosCopia

### Tabla proyeccionesCopia

### Tabla divipolaCopia

### Tabla pibCopia

## Unicidad

### Tabla aeropuertosCopia

In [23]:
print(f"registros = {movimientos.count()}", f"registros distintos = {movimientos.distinct().count()}", f"registros repetidos = {movimientos.count() - movimientos.distinct().count()}")

registros = 204292 registros distintos = 173659 registros repetidos = 30633


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [24]:
movimientos.select(col('TransaccionProductoID')).distinct().count()

173659

De esta manera vemos que no hay  diferencia entre el numero de registros y el numero de registros cuyo identificador es unico.

### Tabla vuelosCopia

### Tabla proyeccionesCopia

### Tabla divipolaCopia

### Tabla pibCopia

## Consistencia
A nivel de consistencia algunos ejemplos de revisión son precios negativos, productos que aparecen en las ordenes pero que no esten registrados en la tabla de produtos. En nuestro caso vamos a verificar que todas las ordenes tengan detalle de orden y viceversa

### Tabla aeropuertosCopia

In [26]:
TransaccionProductoIDS = set([x.TransaccionProductoID for x in movimientos.select('TransaccionProductoID').collect()])
ProveedorIDS = set([x.ProveedorID for x in movimientos.select('ProveedorID').collect()])
OrdenDeCompraIDS = set([x.OrdenDeCompraID for x in movimientos.select('OrdenDeCompraID').collect()])

[len(TransaccionProductoIDS-ProveedorIDS), len(TransaccionProductoIDS-OrdenDeCompraIDS)], [len(ProveedorIDS-TransaccionProductoIDS), len(OrdenDeCompraIDS-TransaccionProductoIDS)]

([173659, 173659], [4, 1472])

Como se puede observar hay una diferencia de 173.659 movimientos que no tienen ProveedorID ni OrdenDeCompraID y hay 4  proveedores que no tienen un movimiento asociado, asi como 1.472 ordenes de compra sin algun movimiento. Ninguno de los casos tiene sentido en el contexto de WideWorldImporters.

### Tabla vuelosCopia

### Tabla proyeccionesCopia

### Tabla divipolaCopia

### Tabla pibCopia

## Validez

### Tabla aeropuertosCopia

In [27]:
CantidadNegativa = [x.Cantidad for x in movimientos.select('Cantidad').collect() if x.Cantidad < 0]
len(CantidadNegativa)

197158

Ya  evidenciamos varios problemas de validez en las  secciones de unicidad y de completitud. Ahora bien, tenemos que hay 197.158 cantidades negativas lo que es un absurdo, la vble cantidad no deberia almacenar numeros menores que cero.

### Tabla vuelosCopia

### Tabla proyeccionesCopia

### Tabla divipolaCopia

### Tabla pibCopia

# Conclusiones/resultados

## ¿Es posible resolver los análisis basados en tableros de control propuestos?

In [28]:
data = movimientos.where((movimientos['ProveedorID'] != '') & (movimientos['OrdenDeCompraID'] != '') & (movimientos['Cantidad'] > 0))
data.show(5)

+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|TransaccionProductoID|ProductoID|TipoTransaccionID|ClienteID|InvoiceID|ProveedorID|OrdenDeCompraID|FechaTransaccion|Cantidad|
+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|               197005|        80|               11|      0.0|      0.0|        4.0|         1272.0|     Feb 10,2015| 11148.0|
|               111093|        95|               11|      0.0|      0.0|        4.0|          745.0|     Mar 27,2014|  7764.0|
|               329831|       193|               11|      0.0|      0.0|        7.0|         2040.0|     May 11,2016| 65520.0|
|               169041|       184|               11|      0.0|      0.0|        7.0|         1100.0|     Oct 29,2014|  8775.0|
|               227339|        98|               11|      0.0|      0.0|        4.0|         1442.0|     May 22

In [29]:
data.count()

7110

In [30]:
print(f"porcentajen datos aptos = {(data.count()/movimientos.count())*100}")

porcentajen datos aptos = 3.4803124938813075


La cantidad de datos optimos para hacer el tablero es de 7.110, lo que implica que el analisis se realizaria sobre el 3.4% de los datos entregados. Siendo esta una muestra muy corta y poco representativa, concluyo que no es recomendable hacer el analisis basado en el tablero de control.

## ¿Que representa la fila promedio de los datos analizados?

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.